In [12]:
from youtube_transcript_api import YouTubeTranscriptApi

import subprocess,whisper ,tqdm , os ,math,warnings ,torch
from faster_whisper import WhisperModel

warnings.filterwarnings("ignore")
from yt_dlp import YoutubeDL

from pathlib import Path 

In [2]:
def download_audio_with_ytdlp(url, output_path="audio.%(ext)s"):
    def progress_hook(d):
        if d['status'] == 'downloading':
            downloaded = d.get('_percent_str', '').strip()
            speed = d.get('_speed_str', '').strip()
            eta = d.get('_eta_str', '').strip()
            print(f"Downloading: {downloaded} | Speed: {speed} | ETA: {eta}", end='\r')
        elif d['status'] == 'finished':
            print(f"\nDownload complete: {d['filename']}")

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_path,  # e.g., "audio.%(ext)s"
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '0',
        }],
        'progress_hooks': [progress_hook],
        'quiet': True,
        'noplaylist': True
    }

    print("Starting download...")
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)

    # Get duration in seconds and convert to mm:ss
    duration_sec = info.get('duration')
    minutes = duration_sec // 60
    seconds = duration_sec % 60
    print(f"\n🔊 Audio duration: {int(minutes)}m {int(seconds)}s")

    return output_path.replace('%(ext)s', 'mp3')




In [ ]:
def audio_to_text(path):
    if torch.cuda.is_available():
        model = WhisperModel("tiny.en", device="cuda", compute_type="float16")
        print("Using GPU for transcription")
    else:
        model = WhisperModel("tiny.en", device="cpu", compute_type="int8")
        print("Using CPU for transcription")
    print("model loaded ")
    segments,_= model.transcribe(language="en",audio=path,chunk_length=120,temperature=0)
    
    final_text=""
    for i, seg in enumerate(tqdm.tqdm(segments, desc="Processing Segments")):
        final_text += seg.text + "\n"
        
    with open("transcript.txt", "w", encoding="utf-8") as f:
        f.write(final_text)

In [4]:

def get_audio_duration(filepath):
    audio = whisper.load_audio(filepath)
    return len(audio) / whisper.audio.SAMPLE_RATE  # SAMPLE_RATE = 16000


In [5]:
# problem :  just taking  longer time ,show demo of 5 minute videos 

In [6]:
download_audio_with_ytdlp("https://www.youtube.com/watch?v=sEqOSvNN4NQ")

Starting download...
[download] 100% of    7.66MiB in 00:00:00 at 9.22MiB/s   Downloading: 0.0% | Speed: Unknown B/s | ETA: 00:12Downloading: 0.0% | Speed: 838.13KiB/s | ETA: 00:09Downloading: 0.1% | Speed: 1.51MiB/s | ETA: 00:05Downloading: 0.2% | Speed: 1.95MiB/s | ETA: 00:03Downloading: 0.4% | Speed: 3.55MiB/s | ETA: 00:02Downloading: 0.8% | Speed: 6.46MiB/s | ETA: 00:01Downloading: 1.6% | Speed: 9.90MiB/s | ETA: 00:00Downloading: 3.3% | Speed: 10.69MiB/s | ETA: 00:00Downloading: 6.5% | Speed: 10.78MiB/s | ETA: 00:00Downloading: 13.0% | Speed: 11.06MiB/s | ETA: 00:00Downloading: 26.1% | Speed: 10.91MiB/s | ETA: 00:00Downloading: 52.2% | Speed: 10.38MiB/s | ETA: 00:00Downloading: 100.0% | Speed: 10.09MiB/s | ETA: 00:00
Download complete: audio.webm
                                                      
🔊 Audio duration: 9m 41s


'audio.mp3'

In [14]:
get_audio_duration("audio.mp3")

580.7573125

In [15]:
audio_to_text("audio.mp3")

model loaded 


Processing Segments: 127it [00:44,  2.85it/s]


In [ ]:
## segments :  taking longer time , show demo of 5 minute videos
## cuda is available in streamlit (try it ): https://discuss.streamlit.io/t/sentence-transformers-using-cpu-on-local-machine-while-using-streamlit-despite-having-gpu-enabled/39185/4